Now we import all necessary libraries

In [5]:
import cv2
import numpy as np
import glob

Variable debug to see intermediate results

In [6]:
debug = False

Now we define all needed variables and types of points

In [7]:
# define chessboard with dimensions 9x6
chessboardDimensions = (6 , 9)

# criteria to stop iterations with max number and epsilon value
maximumIterationsNumber = 30
epsilon = 0.001
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER , maximumIterationsNumber , epsilon)

# set of 2d points
twoDPoints = [ ]

# set of 3d points
threeDPoints = [ ]

# set of 3d real world coordinates using the dimensions defined before
threeDIncrementalPoints = np.zeros ( (1 , chessboardDimensions [ 0 ] * chessboardDimensions [ 1 ] , 3) , np.float32 )

# prepare threeDRealWorldPoints using this format [[[0,0,0] [1,0,0] ..... [5. 0. 0.]] \n [[0,1,0] [1,1,0] ..... [5. 1. 0.]] \n ..... \n [[0,8,0] [1,8,0] ..... [5. 8. 0.]]]
threeDIncrementalPoints [ 0 , : , :2 ] = np.mgrid [ 0 :chessboardDimensions [ 0 ] , 0 :chessboardDimensions [ 1 ] ].T.reshape ( -1 , 2 )

In [8]:
# get list of calibration images
listImages = glob.glob ( "framesVideoCalibration/uncalibrated/*.jpg" )

# index of actual image in the list starting from 1 to n
indexImageInList = 1

# get height of image
height = 0

# get width of image
width = 0

# for each image in the list
for filename in listImages :

	# read the actual image
	image = cv2.imread ( filename )

	# if the actual image is the first in the list
	if indexImageInList == 1 :

		# get height and width of the image (we consider that all frames in the video has same height and width)
		height , width = image.shape [ :2 ]

	# transform the actual image in gray scale (cv2 use bgr)
	imageGreyColor = cv2.cvtColor ( image , cv2.COLOR_BGR2GRAY )

	# try to find all chessboard corners
	ret , corners = cv2.findChessboardCorners ( imageGreyColor , chessboardDimensions , cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE )

	# if the corners are found
	if ret :

		# append the incremental points in the 3d points list
		threeDPoints.append ( threeDIncrementalPoints )

		# refining corners coordination's
		corners = cv2.cornerSubPix ( imageGreyColor , corners , (11 , 11) , (-1 , -1) , criteria )

		# append corners coordination's in the 2d points list
		twoDPoints.append ( corners )

		# index for the corners
		cornerIndex = 1

		# for each corner
		for corner in corners :

			# calculate the integer coordinates for the actual corner
			coordinatesActualPoint = (int ( corner [ 0 ] [ 0 ] ) , int ( corner [ 0 ] [ 1 ] ))

			# plot a circle in the actual corner
			imageWithCorners = cv2.circle ( image , coordinatesActualPoint , radius = 4 , color = (0 , 255 , 255) , thickness = -1 )

			# add the index n the actual corner
			imageWithCornersAndTexts = cv2.putText ( imageWithCorners , str ( cornerIndex ) , coordinatesActualPoint , cv2.FONT_HERSHEY_SIMPLEX , 1 , (0 , 0 , 255) , 2 )

			# increment the index for the next indexes
			cornerIndex += 1

		# save actual image in calibrated folder with corrected path
		cv2.imwrite ( filename.replace ( "uncalibrated" , "calibrated" ) , imageWithCornersAndTexts )

	if debug :

		# show the actual image with index
		cv2.imshow ( "Calibrated image with index" , imageWithCornersAndTexts )

		# wait the key 0 to go forward
		cv2.waitKey ( 0 )

	# increment index in the list
	indexImageInList = indexImageInList + 1

# as last thing destroy all windows
cv2.destroyAllWindows ( )

# calibrate camera
ret , K , dist , rvecs , tvecs = cv2.calibrateCamera ( threeDPoints , twoDPoints , imageGreyColor.shape [ : :-1 ] , None , None )

# Displaying required output
print ( " Camera matrix:" )
print ( K )

print ( "\n Distortion coefficient:" )
print ( dist )

print ( "\n Rotation Vectors:" )
print ( rvecs )

print ( "\n Translation Vectors:" )
print ( tvecs )

# save intrinsics
Kfile = cv2.FileStorage ( "framesVideoCalibration/calibrated/intrinsics.xml" , cv2.FILE_STORAGE_WRITE )
Kfile.write ( "K" , K )
Kfile.write ( "dist" , dist )
np.save ( "framesVideoCalibration/uncalibrated/K" , K )
np.save ( "framesVideoCalibration/uncalibrated/dist" , dist )



 Camera matrix:
[[1.66269050e+03 0.00000000e+00 9.57357108e+02]
 [0.00000000e+00 1.66468209e+03 5.28170272e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]

 Distortion coefficient:
[[ 1.08397912e-01 -6.59296999e-03 -7.22760934e-06 -1.57363467e-03
  -2.96117308e-01]]

 Rotation Vectors:
(array([[-0.08162514],
       [ 0.44945738],
       [-0.13847282]]), array([[-0.67073169],
       [-0.02278574],
       [-2.90560473]]), array([[0.65910535],
       [0.12335504],
       [2.83303457]]), array([[0.55813817],
       [0.220439  ],
       [2.4389295 ]]), array([[0.51885405],
       [0.29397403],
       [2.13822145]]), array([[0.42840587],
       [0.33858762],
       [1.79267251]]), array([[0.48079387],
       [0.48142243],
       [1.53662622]]), array([[0.43071842],
       [0.47894962],
       [1.54342841]]), array([[0.19946234],
       [0.23886343],
       [1.65776162]]), array([[0.26388862],
       [0.21951384],
       [2.34381936]]), array([[0.44533104],
       [0.22990111],
       [